# Ejemplo de pruebas con Backtrader

Este es un simple ejemplo como realizar pruebas con backtesting para demostrar su funcionamiento.

Una explicación más completa del código se puede ver en el blog:
http://www.garcia-ferreira.es/backtesting-con-backtrader

# Instalando las librerías necesarias

In [ ]:
!pip install backtrader

In [ ]:
!pip install yahoo_fin

# Importando librerías

In [ ]:
import backtrader as bt

import datetime
import pytz

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.font_manager

import requests
import pandas as pd

# Descargando los datos

In [ ]:
from yahoo_fin.stock_info import get_data
starbucks = get_data("sbux", start_date="01/01/2015", end_date="01/01/2022", index_as_date=True, interval="1d")

# Clase principal de la estrategia

In [ ]:
class cruce50_200(bt.Strategy):
  def __init__(self):
    # Inicializamos la media de 50 y la de 200
    self.sma50 = bt.indicators.SimpleMovingAverage(self.data, period=50, plotname="50 SMA")
    self.sma200 = bt.indicators.SimpleMovingAverage(self.data, period=200, plotname="200 SMA")
    self.cruce = bt.ind.CrossOver(self.sma50,self.sma200)

    # Inicializamos la variable order, que usaremos para el control de las ordenes
    # Para mantener el seguimiento de las órdenes pendientes
    self.order = None
    self.buyprice = None
    self.sellprice = None

    # guardamos el último dato de cierre
    self.dataclose = self.datas[0].close

  def next(self):
    # Comprobamos si estamos en el mercado
    # si no lo estamos seguimos para adelante
    if not self.position:
      # Si la media de 50 es mayor que la media de 200 compramos
      # print("{}: {} - {}".format(self.datas[0].datetime.date(0), self.sma50[0], self.sma200[0]))
      if self.cruce > 0:
        self.order = self.buy()
        self.log('Orden de compra lanzada: %.2f' % self.dataclose[0])
    # Si estamos en el mercado
    else:
      if self.cruce < 0:
          # Si es así vendemos (con todos los parametros por defecto posibles)
          self.log('Orden de venta lanzada: %.2f' % self.dataclose[0])
          
          # Mantenemos un seguimiento de la orden para evitar abrir una segunda orden
          self.order = self.sell()

  # Creamos este método para el control de las ordenes
  def notify_order(self, order):
    # Orden de compra o de venta aceptada por el brocker - Nada que hacer
    if order.status in [order.Submitted, order.Accepted]:
      return
    # Orden de compra completada
    # Comprobamos si es de compra o de venta y mostramos los resultados
    if order.status in [order.Completed]:
      if order.isbuy():
        self.buyprice = order.executed.price
        self.log('COMPRA EJECUTADA [Precio: %.2f, Comisión: %.2f]' % (order.executed.price, order.executed.comm))
      elif order.issell():
        self.sellprice = order.executed.price
        self.log('VENTA EJECUTADA [Precio: %.2f, Comisión: %.2f]' % (order.executed.price,  order.executed.comm))
    # Orden de compra cancelada
    elif order.status == order.Canceled:
      self.log('Orden Cancelada')
    # Orden de compra cancelada por el margen de tu dinero
    elif order.status == order.Margin:
      self.log('Orden de Margen')
    # Orden de compra rechazada
    elif order.status == order.Rejected:
      self.log('Orden Rechazada')

  # Metodo para mostrar de manera más fácil los logs por pantalla
  # incluyendo la fecha en la que se producen los eventos
  def log(self, txt, dt=None):
        # Vamos mostrando los datos
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

# Código principal de backtrader

In [ ]:
cerebro = bt.Cerebro()

# Añadimos los datos que hemso descargado previamente
data1 = bt.feeds.PandasData(dataname=starbucks)
cerebro.adddata(data1)

# Añadimos la cantidad inicial de dinero con la que vamos a realizar el trading
cerebro.broker.setcash(10000.0)
# Añadimos la comisión - 0.1%
cerebro.broker.setcommission(commission=0.001)
# Tamaño de los lotes que queremos comprar
cerebro.addsizer(bt.sizers.FixedSize, stake=1)

cerebro.addstrategy(cruce50_200)

# Mostramos los valores tanto inicial como final durante la ejecución del
# proceso de backtesting
print('Valor inicial del portfolio: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Valor final del porfolio: %.2f' % cerebro.broker.getvalue())

# Mostrando los datos gráficamente


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# plt.rcParams['font.sans-serif']=['SimHei']
# plt.rcParams['axes.unicode_minus']=False
plt.rcParams['figure.figsize']=[40, 16]
plt.rcParams['figure.dpi']=500
# plt.rcParams['figure.facecolor']='w'
plt.rcParams['figure.edgecolor']='k'
cerebro.plot(style='candle', iplot=False, volume = True, barupfill = False, bardownfill = False, barup='green', bardown='red')